In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import datetime

def strptime_date(df,pattern='%Y-%m-%d %H:%M:%S'):
    return [datetime.datetime.strptime(df[i], pattern) for i in range(len(df))]

marged_df=pd.read_csv('/content/drive/MyDrive/signate/シェアサイクルの利用予測/marged_df.csv')
marged_df["date"]=strptime_date(marged_df["date"])
marged_df["installation_date"]=strptime_date(marged_df["installation_date"])
marged_df.head()

,id,year,month,day,hour,station_id,bikes_available,predict,date,rent_num,return_num,weather1,weather2,lat,long,dock_count,city,installation_date,is_install
0,0.0,2013.0,9.0,1.0,0.0,0.0,11.0,0.0,2013-09-01 00:00:00,0.0,0.0,-2.753002,-1.330403,37.32973,-121.90178,27,city1,2013-08-06,True
1,1.0,2013.0,9.0,1.0,1.0,0.0,11.0,0.0,2013-09-01 01:00:00,0.0,0.0,-2.753002,-1.330403,37.32973,-121.90178,27,city1,2013-08-06,True
2,2.0,2013.0,9.0,1.0,2.0,0.0,11.0,0.0,2013-09-01 02:00:00,0.0,0.0,-2.753002,-1.330403,37.32973,-121.90178,27,city1,2013-08-06,True
3,3.0,2013.0,9.0,1.0,3.0,0.0,11.0,0.0,2013-09-01 03:00:00,0.0,0.0,-2.753002,-1.330403,37.32973,-121.90178,27,city1,2013-08-06,True
4,4.0,2013.0,9.0,1.0,4.0,0.0,11.0,0.0,2013-09-01 04:00:00,0.0,0.0,-2.753002,-1.330403,37.32973,-121.90178,27,city1,2013-08-06,True


station_idごとに求める　過去28日のデータ使用

In [ ]:
station_num=70
dfs=[0]*station_num
is_predict=[0]*station_num
for i in range(station_num):
    tmp_df=marged_df.loc[(marged_df["station_id"]==i)&(marged_df["is_install"]==True),["bikes_available","predict","weather1","weather2"]].reset_index(drop=True)
    dfs[i]=tmp_df.loc[:,["bikes_available","weather1","weather2"]]
    is_predict[i]=tmp_df.loc[:,"predict"]

In [ ]:
from scipy import stats
import numpy as np

each_mean=[0]*station_num
each_std=[0]*station_num
data=[0]*station_num

for i in range(station_num):
    tmp=dfs[i].to_numpy(dtype=float)
    each_mean=tmp.mean(axis=0)
    each_std=np.std(tmp, axis=0)
    data[i]=stats.zscore(tmp)

In [ ]:
import torch
margin=28*24
X_len=len(data[0])-margin
X=[[0]*X_len]*station_num
Y=[[0]*X_len]*station_num

for i in range(station_num):
    tmp=data[i].transpose()
    tmp_X=tmp[1:3].transpose()
    tmp_Y=tmp[0].transpose()
    for j in range(len(tmp_X)-margin):
        X[i][j]=tmp_X[j:j+margin]
        Y[i][j]=tmp_Y[j:j+margin]

In [ ]:
import torch
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset
from sklearn.model_selection import train_test_split
def make_dataset(X,Y,station_id):
    x=torch.FloatTensor(X[station_id])
    y=torch.FloatTensor(Y[station_id])
    train_x,test_x,train_y,test_y = train_test_split(x,y,test_size = 0.1)
    train_dataset=TensorDataset(train_x,train_y)
    train_loader=DataLoader(train_dataset)
    test_dataset=TensorDataset(test_x,test_y)
    test_loader=DataLoader(test_dataset)
    return (train_loader,test_loader)
train_loader,test_loader=make_dataset(X,Y,0)

In [ ]:
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

class RNN(nn.Module):
    def __init__(self, input_size,hidden_layer_size,output_size):
        super().__init__()
        self.rnn = nn.GRU(input_size, hidden_layer_size,batch_first=True)
        self.fc = nn.Linear(hidden_layer_size, output_size)

    def forward(self, x):
        x=x.to(device)
        print(x.size())
        x,_= self.rnn(x)
        print(x.size())
        print(x[:, -1, :])
        x= self.fc(x[:, -1, :])
        return x

cpu


In [ ]:

from numpy.ma.core import count
from datetime import datetime
from pytz import timezone
import matplotlib.pyplot as plt

model = RNN(2,margin,1).to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01)
criterion = nn.MSELoss()

epoch_num=50
train_loss_list=[]*epoch_num

model.train()
for i in range(epoch_num):
    total_loss = 0
    for x,y in train_loader:
        optimizer.zero_grad()
        pred_y = model(x)
        loss = criterion(pred_y, y)
        loss.backward()
        optimizer.step()
        total_loss+=loss.item()
    if i%10==0:
        print("epoch "+str(i)+" "+datetime.now(timezone('Asia/Tokyo')).strftime('%m/%d %H:%M:%S')+" total_loss:"+str(total_loss))
        train_loss_list.append(total_loss)
plt.plot(train_loss_list)
plt.show()
#
model.eval()
pred_y_list=[]
y_list=[]
with torch.no_grad(): 
    for x,y in test_loader:
        pred_y = model(x)
        y_list.append(y.item())
        pred_y_list.append(pred_y.item())

In [ ]:

criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)



record_loss_train = []*epoch_num

model.train()
for i in range(epoch_num):
    total_loss = 0
    for x,y in train_loader:
        optimizer.zero_grad()
        pred_y = model(x)
        loss = criterion(pred_y, y)
        loss.backward()
        optimizer.step()
        total_loss+=loss.item()
    if i%10==0:
        print("epoch "+str(i)+" "+datetime.now(timezone('Asia/Tokyo')).strftime('%m/%d %H:%M:%S')+" total_loss:"+str(total_loss))
        loss_list.append(total_loss)
plt.plot(loss_list)
plt.show()
#
model.eval()
pred_y_list=[]
y_list=[]
with torch.no_grad(): 
    for x,y in test_loader:
        pred_y = model(x)
        y_list.append(y.item())
        pred_y_list.append(torch.argmax(pred_y).item())

for i in range(201):
  model.train()
  loss_train = 0
  for j, (x, t) in enumerate(train_loader):
    loss = criterion(model(x), t.to(device))
    loss_train += loss.item()
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
  loss_train /= j+1
  record_loss_train.append(loss_train)
  if i%10 == 0:
    print("Epoch:", i, "Loss_Train:", loss_train)
    predicted = list(input_data[0].reshape(-1))
    model.eval()
    with torch.no_grad():
      for i in range(n_sample):
        x = torch.tensor(predicted[-n_time:])
        x = x.reshape(1, n_time, 1)
        predicted.append(model(x)[0].item())
    plt.plot(range(len(sin_x)), sin_x, label="Correct")
    plt.plot(range(len(predicted)), predicted, label="Predicted")
    plt.legend()
    plt.show()